In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


     |████████████████████████████████| 1.1MB 9.4MB/s 
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [2]:
keras.backend.clear_session()
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
len(shakespeare_text)

1115394

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [5]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [7]:
max_id = len(tokenizer.word_index)

In [8]:
max_id

39

In [9]:
tokenizer.word_index

{'\n': 11,
 ' ': 1,
 '!': 31,
 '$': 39,
 '&': 38,
 "'": 28,
 ',': 18,
 '-': 32,
 '.': 27,
 '3': 37,
 ':': 24,
 ';': 29,
 '?': 30,
 'a': 5,
 'b': 22,
 'c': 19,
 'd': 13,
 'e': 2,
 'f': 20,
 'g': 21,
 'h': 7,
 'i': 6,
 'j': 33,
 'k': 25,
 'l': 12,
 'm': 15,
 'n': 10,
 'o': 4,
 'p': 23,
 'q': 34,
 'r': 9,
 's': 8,
 't': 3,
 'u': 14,
 'v': 26,
 'w': 17,
 'x': 35,
 'y': 16,
 'z': 36}

In [10]:
dataset_size = tokenizer.document_count

In [11]:
dataset_size

1

In [12]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [13]:
len(encoded)

1115394

In [14]:
type(encoded)

numpy.ndarray

In [15]:
dataset_size = len(encoded)

In [16]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [17]:
dataset
i = 0
for item in dataset:
  i += 1
print(i)

1003854


In [18]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [19]:
dataset

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>

In [20]:
for item in dataset.take(3):
  print(item)

<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>


In [21]:
i = 0
for item in dataset.take(1):
  for item1 in item:
    print(item1)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(35, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(23, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(21, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(16, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(22, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(3, shape=

In [22]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [23]:
dataset

<FlatMapDataset shapes: (None,), types: tf.int64>

In [24]:
for item in dataset.take(3):
  print(item)

tf.Tensor(
[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13  0], shape=(101,), dtype=int64)
tf.Tensor(
[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24 17
  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10
 15  3 13  0  4], shape=(101,), dtype=int64)
tf.Tensor(
[ 8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22
  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8
  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24 17  0
  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 15
 

In [25]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)

In [26]:
for item in dataset.take(1):
  print(item)

tf.Tensor(
[[ 6  5  7 ...  0 18  4]
 [ 6  1  0 ...  0 11  1]
 [11 12  0 ... 29 10 10]
 ...
 [ 3 27  0 ...  7  3 19]
 [18  3  9 ...  4  9  7]
 [14  7  0 ...  8  1  0]], shape=(32, 101), dtype=int64)


In [27]:
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [28]:
for item in dataset.take(1):
  print(item)

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[13,  0,  2, ...,  4, 14,  5],
       [ 0,  9,  1, ...,  9,  1,  9],
       [ 3,  0,  2, ...,  0,  3, 13],
       ...,
       [19,  5,  9, ...,  8,  1,  0],
       [ 0, 18,  5, ...,  0, 20,  3],
       [ 6,  5,  7, ...,  2,  6,  5]])>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 0,  2,  6, ..., 14,  5,  9],
       [ 9,  1,  5, ...,  1,  9,  5],
       [ 0,  2,  6, ...,  3, 13,  8],
       ...,
       [ 5,  9, 12, ...,  1,  0, 13],
       [18,  5,  2, ..., 20,  3,  3],
       [ 5,  7,  0, ...,  6,  5,  9]])>)


In [29]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [30]:
for item in dataset.take(1):
  print(item)

(<tf.Tensor: shape=(32, 100, 39), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.

In [31]:
dataset = dataset.prefetch(1)

In [32]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                      dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                      dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])


In [33]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [34]:
#history = model.fit(dataset,  steps_per_epoch=train_size // batch_size,
#                    epochs=5)

In [35]:
tf.random.set_seed(42)
batch_size = 1

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)


In [36]:
n_steps

100

In [37]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [38]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [39]:
#model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
#steps_per_epoch = train_size // batch_size // n_steps
#model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
#                   callbacks=[ResetStatesCallback()])


In [40]:
# Sentiment Analysis

In [41]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 0s 0us/step


In [42]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [43]:
word_index = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [44]:
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

In [45]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_] = token

In [46]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [47]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCA66SV/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCA66SV/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCA66SV/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [48]:
datasets

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [49]:
info.name

'imdb_reviews'

In [50]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [51]:
for item in datasets['test'].take(2):
  print(item)

(<tf.Tensor: shape=(), dtype=string, numpy=b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.">, <tf.Tensor: 

In [52]:
for item in datasets['unsupervised'].take(2):
  print(item[0])

tf.Tensor(b"SPOILER - Now knowing the ending I find it so clever that the whole movie takes place in a motel and each character has a different room. Even sane people have many different aspects to their personality, but they don't let them become dominant -- they are controlled. Malcolm's various personalities and needs were personified in each character. The prostitute mother (Amanda Peet), the part of him who hated her for being a prostitute (Larry), the loving mother he wish he had, the loving father he wish he had, the selfish part of himself (actress), the violent part of his personality (Ray Liotta and Busey), the irrational emotions he feels and his need to be loved (Ginnie) and his attempts to control those feelings (Lou), the hurt little boy who sees far too many traumatic things in his life, and of course, John Cusack who seems to represent Malcolm himself trying to analyze and understand all the craziness in his mind, tries to follow the rules (accepting responsibility for 

In [53]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [54]:
info.splits

{'test': <tfds.core.SplitInfo num_examples=25000>,
 'train': <tfds.core.SplitInfo num_examples=25000>,
 'unsupervised': <tfds.core.SplitInfo num_examples=50000>}

In [55]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()



Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [56]:
for item in datasets["train"].batch(2).take(1):
  print(item)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [57]:
datasets["train"]

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [58]:
datasets["train"]

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [59]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch):
        print("Review:", review[:200])


Review: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting "
Review: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However '


In [60]:
datasets["train"]

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [61]:
a = np.array([1,2,3,4])

In [62]:
at = tf.convert_to_tensor(a)

In [63]:
ats = tf.data.Dataset.from_tensor_slices(at)

In [64]:
k = ats.batch(2)

In [65]:
for item in k:
  print(item)

tf.Tensor([1 2], shape=(2,), dtype=int64)
tf.Tensor([3 4], shape=(2,), dtype=int64)


In [66]:
for i,j in ([1,2],[2,3]):
  print (i,j)

1 2
2 3


In [67]:
X_batch

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [68]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [69]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
  print(X_batch,y_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [70]:
X_batch

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [71]:
X_batch1 = tf.strings.substr(X_batch, 0, 300)

In [72]:
X_batch1

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda pi",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Cons'],
      dtype=object)>

In [73]:
X_batch2 = tf.strings.regex_replace(X_batch1, b"<br\\s*/?>", b" ")

In [74]:
X_batch2

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda pi",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Cons'],
      dtype=object)>

In [75]:
X_batch3 = tf.strings.regex_replace(X_batch2, b"[^a-zA-Z']", b" ")

In [76]:
X_batch3

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie  Don't be lured in by Christopher Walken or Michael Ironside  Both are great actors  but this must simply be their worst role in history  Even their great acting could not redeem this movie's ridiculous storyline  This movie is an early nineties US propaganda pi",
       b'I have been known to fall asleep during films  but this is usually due to a combination of things including  really tired  being warm and comfortable on the sette and having just eaten a lot  However on this occasion I fell asleep because the film was rubbish  The plot development was constant  Cons'],
      dtype=object)>

In [77]:
X_batch4 = tf.strings.split(X_batch3)

In [78]:
X_batch4

<tf.RaggedTensor [[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie', b"Don't", b'be', b'lured', b'in', b'by', b'Christopher', b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are', b'great', b'actors', b'but', b'this', b'must', b'simply', b'be', b'their', b'worst', b'role', b'in', b'history', b'Even', b'their', b'great', b'acting', b'could', b'not', b'redeem', b'this', b"movie's", b'ridiculous', b'storyline', b'This', b'movie', b'is', b'an', b'early', b'nineties', b'US', b'propaganda', b'pi'], [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep', b'during', b'films', b'but', b'this', b'is', b'usually', b'due', b'to', b'a', b'combination', b'of', b'things', b'including', b'really', b'tired', b'being', b'warm', b'and', b'comfortable', b'on', b'the', b'sette', b'and', b'having', b'just', b'eaten', b'a', b'lot', b'However', b'on', b'this', b'occasion', b'I', b'fell', b'asleep', b'because', b'the', b'film', b'was', b'rubbish', b'The', b'plot', b'development', b'was'

In [79]:
X_batch4.to_tensor()

<tf.Tensor: shape=(2, 53), dtype=string, numpy=
array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
        b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
        b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
        b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
        b'their', b'worst', b'role', b'in', b'history', b'Even',
        b'their', b'great', b'acting', b'could', b'not', b'redeem',
        b'this', b"movie's", b'ridiculous', b'storyline', b'This',
        b'movie', b'is', b'an', b'early', b'nineties', b'US',
        b'propaganda', b'pi', b'', b'', b''],
       [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
        b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
        b'to', b'a', b'combination', b'of', b'things', b'including',
        b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
        b'on', b'the', b'sette', b'and', b'having', b'just', b'eaten',
        b'a', 

In [80]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [81]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [82]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [83]:
truncated_vocabulary

[b'<pad>',
 b'the',
 b'a',
 b'of',
 b'and',
 b'to',
 b'I',
 b'is',
 b'in',
 b'this',
 b'it',
 b'was',
 b'movie',
 b'that',
 b'The',
 b'film',
 b'with',
 b'for',
 b'as',
 b'on',
 b'but',
 b'have',
 b'This',
 b'one',
 b'not',
 b'be',
 b'are',
 b'you',
 b'an',
 b'at',
 b'about',
 b'by',
 b'all',
 b'his',
 b'so',
 b'like',
 b'from',
 b'who',
 b'has',
 b'It',
 b'good',
 b'my',
 b'just',
 b'very',
 b'out',
 b'or',
 b'story',
 b'some',
 b'time',
 b'had',
 b'he',
 b'they',
 b'really',
 b'me',
 b'when',
 b'what',
 b'first',
 b'movies',
 b'bad',
 b'see',
 b'seen',
 b'up',
 b'only',
 b'were',
 b"it's",
 b'would',
 b'more',
 b'made',
 b'great',
 b'can',
 b'been',
 b'i',
 b'her',
 b'no',
 b'A',
 b'which',
 b'even',
 b'films',
 b'there',
 b'ever',
 b'people',
 b'much',
 b'because',
 b'most',
 b'plot',
 b'if',
 b'than',
 b'acting',
 b'get',
 b'their',
 b'well',
 b'into',
 b'how',
 b'best',
 b'think',
 b'other',
 b'its',
 b"It's",
 b'saw',
 b'could',
 b'watch',
 b'many',
 b"don't",
 b'do',
 b'will',
 

In [84]:
tf.strings.substr([b'Hello', b'World'],1,2)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'el', b'or'], dtype=object)>

In [85]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [86]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [87]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [88]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 21s 26ms/step - loss: 0.6320 - accuracy: 0.6060
Epoch 2/5
782/782 [==============================] - 21s 26ms/step - loss: 0.3968 - accuracy: 0.8233
Epoch 3/5
782/782 [==============================] - 21s 26ms/step - loss: 0.2377 - accuracy: 0.9077
Epoch 4/5
782/782 [==============================] - 20s 26ms/step - loss: 0.1560 - accuracy: 0.9436
Epoch 5/5
782/782 [==============================] - 20s 26ms/step - loss: 0.1187 - accuracy: 0.9566
